# Merging and cleaning overall audio feature df

In [2]:
# impoarting needed libraries

import pandas as pd

In [3]:
# importing both audio feature data files

audio_features_top100 = pd.read_csv('../data/top100_audio_features.csv')
audio_features_dr_sample = pd.read_csv('../data/dr_sample_audio_features.csv')

In [4]:
# checking columns top100

audio_features_top100.columns

# need to remove the Unnamed

Index(['Unnamed: 0', 'id', 'track_title', 'track_explicit', 'track_artist',
       'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
       'type', 'uri', 'track_href', 'analysis_url', 'duration_ms',
       'time_signature'],
      dtype='object')

In [5]:
# checking columns dr sample

audio_features_dr_sample.columns

# need to remove the Unnamed and 0 column

Index(['Unnamed: 0', 'id', 'track_title', 'track_explicit', 'track_artist',
       '0', 'acousticness', 'analysis_url', 'danceability', 'duration_ms',
       'energy', 'instrumentalness', 'key', 'liveness', 'loudness', 'mode',
       'speechiness', 'tempo', 'time_signature', 'track_href', 'type', 'uri',
       'valence'],
      dtype='object')

In [6]:
len(audio_features_top100.columns), len(audio_features_dr_sample.columns)

(22, 23)

to dos:
- remove Unnamed column from both df
- remove 0 column from dr df
- rearrange df dr columns to same order as top100 dr
- add column data source (dr sample, top100)
- add genre in both
- add release date (album)
- concat both dfs to one

In [7]:
# dropping not needed columns

audio_features_top100 = audio_features_top100.drop(['Unnamed: 0'], axis = 1)
audio_features_dr_sample = audio_features_dr_sample.drop(['Unnamed: 0'], axis = 1)
audio_features_dr_sample = audio_features_dr_sample.drop(['0'], axis = 1)

In [8]:
# adding column datasource

audio_features_top100['data_source'] = 'top100'
audio_features_dr_sample['data_source'] = 'dr_sample'

In [9]:
# adding column genre in again dr sample

audio_features_dr_sample['genre'] = 'deutschrap'

In [10]:
# adding column genre in again top100

# importing dr artist data
dr_artists = pd.read_csv('../data/dr_artist_name_id_df.csv')

# creating a list of all unique dr artist names
dr_artists_lst = list(dr_artists['artist_name'].unique())

# importing top20 hip hop data
top20_hiphop = pd.read_csv('../data/top20_hiphop_tracks_raw.csv')

# creating a list of all unique hiphop artist names
top20_hiphop_artists_lst = list(top20_hiphop['artist'].unique())

# importing christmas tracks
christmas_tracks = pd.read_csv('../data/christmas_tracks.csv')

# creating a list of all unique christmas tracks
christmas_tracks_lst = list(christmas_tracks['title'].unique())

audio_features_top100['genre'] = 0

# creating new feature to mark the tracks which are christmas tracks
for track in christmas_tracks_lst:
    for i in range(len(audio_features_top100)):
        if track in audio_features_top100.loc[i,'track_title']:
            audio_features_top100.loc[i, 'genre'] = 'christmas'

# filling empty genre column in top100 with hiphop from artists in top20
for artist in top20_hiphop_artists_lst:
    for i in range(len(audio_features_top100)):
        if artist in audio_features_top100.loc[i,'track_artist']:
            audio_features_top100.loc[i, 'genre'] = 'hiphop'
            
# filling empty genre column in top100 with dr artists from artists in dr

for artist in dr_artists_lst:
    for i in range(len(audio_features_top100)):
        if artist in audio_features_top100.loc[i,'track_artist']:
            audio_features_top100.loc[i, 'genre'] = 'deutschrap'

In [11]:
audio_features_top100.columns

Index(['id', 'track_title', 'track_explicit', 'track_artist', 'danceability',
       'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness',
       'instrumentalness', 'liveness', 'valence', 'tempo', 'type', 'uri',
       'track_href', 'analysis_url', 'duration_ms', 'time_signature',
       'data_source', 'genre'],
      dtype='object')

In [12]:
# rearranging the columns

audio_features_dr_sample = audio_features_dr_sample[['id', 'track_title', 'track_explicit', 'track_artist', 'danceability',
       'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness',
       'instrumentalness', 'liveness', 'valence', 'tempo', 'type', 'uri',
       'track_href', 'analysis_url', 'duration_ms', 'time_signature',
       'data_source', 'genre']]

In [13]:
len(audio_features_top100), len(audio_features_dr_sample)

(17735, 28077)

In [14]:
# concatenating the 2 df

audio_features_all = pd.concat([audio_features_dr_sample, audio_features_top100], axis=0)
len(audio_features_all)

45812

In [16]:
# importing release date data

release_dates = pd.read_csv('../data/tracks_all_release.csv')

In [27]:
len(release_dates)

45812

In [19]:
release_dates.columns

Index(['Unnamed: 0', 'track_id', 'track_album_id', 'track_album_name',
       'track_album_release'],
      dtype='object')

In [28]:
release_dates.columns = ['Unnamed: 0', 'id', 'track_album_id', 'track_album_name',
       'track_album_release']

In [38]:
audio_features_all_new = pd.merge(audio_features_all, release_dates, on='id')

In [39]:
audio_features_all_new.columns

Index(['id', 'track_title', 'track_explicit', 'track_artist', 'danceability',
       'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness',
       'instrumentalness', 'liveness', 'valence', 'tempo', 'type', 'uri',
       'track_href', 'analysis_url', 'duration_ms', 'time_signature',
       'data_source', 'genre', 'release_date', 'Unnamed: 0', 'track_album_id',
       'track_album_name', 'track_album_release'],
      dtype='object')

In [40]:
audio_features_all_new = audio_features_all_new.drop(['release_date', 'Unnamed: 0'], axis = 1)

In [41]:
audio_features_all_new = audio_features_all_new[['id', 'track_title', 'track_artist',
       'track_album_id','track_album_name', 'track_album_release', 'genre', 'track_explicit', 'data_source', 'danceability',
       'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness',
       'instrumentalness', 'liveness', 'valence', 'tempo', 'type', 'uri',
       'track_href', 'analysis_url', 'duration_ms', 'time_signature']]

In [42]:
audio_features_all_new.head()

,id,track_title,track_artist,track_album_id,track_album_name,track_album_release,genre,track_explicit,data_source,danceability,...,instrumentalness,liveness,valence,tempo,type,uri,track_href,analysis_url,duration_ms,time_signature
0,5mrNkGIEJ0pSdXIR0eMwZQ,Bandolero,Azero,157IcJKKD2MNQtqTFy2HgT,Bela,2019-10-31,deutschrap,True,dr_sample,0.726,...,0.000002,0.131,0.318,145.955,audio_features,spotify:track:5mrNkGIEJ0pSdXIR0eMwZQ,https://api.spotify.com/v1/tracks/5mrNkGIEJ0pS...,https://api.spotify.com/v1/audio-analysis/5mrN...,173003.0,4.0
1,1EFbLCDFWIIiC6IMgthYsN,El Niño (Alarm),Azero,157IcJKKD2MNQtqTFy2HgT,Bela,2019-10-31,deutschrap,True,dr_sample,0.765,...,0.000005,0.118,0.541,173.915,audio_features,spotify:track:1EFbLCDFWIIiC6IMgthYsN,https://api.spotify.com/v1/tracks/1EFbLCDFWIIi...,https://api.spotify.com/v1/audio-analysis/1EFb...,143867.0,4.0
2,3czNqQU3YvEcxaxIz16nb7,Es kocht,Azero,157IcJKKD2MNQtqTFy2HgT,Bela,2019-10-31,deutschrap,True,dr_sample,0.551,...,0.000339,0.147,0.158,94.261,audio_features,spotify:track:3czNqQU3YvEcxaxIz16nb7,https://api.spotify.com/v1/tracks/3czNqQU3YvEc...,https://api.spotify.com/v1/audio-analysis/3czN...,196682.0,4.0
3,4oqjF0RgQmtGGogG3paqOT,Barbaren,Azero,157IcJKKD2MNQtqTFy2HgT,Bela,2019-10-31,deutschrap,True,dr_sample,0.708,...,0.000040,0.111,0.359,170.071,audio_features,spotify:track:4oqjF0RgQmtGGogG3paqOT,https://api.spotify.com/v1/tracks/4oqjF0RgQmtG...,https://api.spotify.com/v1/audio-analysis/4oqj...,227509.0,4.0
4,5E3dnmFiGepqED3KpwJalV,Sierra Leone,Azero,157IcJKKD2MNQtqTFy2HgT,Bela,2019-10-31,deutschrap,True,dr_sample,0.724,...,0.000000,0.165,0.554,127.914,audio_features,spotify:track:5E3dnmFiGepqED3KpwJalV,https://api.spotify.com/v1/tracks/5E3dnmFiGepq...,https://api.spotify.com/v1/audio-analysis/5E3d...,130719.0,4.0


In [43]:
# saving df 

audio_features_all_new.to_csv('../data/audio_features_all_V2.csv')

In [44]:
audio_features_all_new.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50622 entries, 0 to 50621
Data columns (total 26 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id                   50622 non-null  object 
 1   track_title          50622 non-null  object 
 2   track_artist         50622 non-null  object 
 3   track_album_id       50622 non-null  object 
 4   track_album_name     50622 non-null  object 
 5   track_album_release  50622 non-null  object 
 6   genre                50622 non-null  object 
 7   track_explicit       50622 non-null  bool   
 8   data_source          50622 non-null  object 
 9   danceability         49882 non-null  float64
 10  energy               49882 non-null  float64
 11  key                  49882 non-null  float64
 12  loudness             49882 non-null  float64
 13  mode                 49882 non-null  float64
 14  speechiness          49882 non-null  float64
 15  acousticness         49882 non-null 

In [53]:
audio_features_all_new['genre'].value_counts()

deutschrap    35110
0             14351
hiphop         1155
christmas         6
Name: genre, dtype: int64

In [50]:
audio_features_all_new['data_source'].value_counts()

dr_sample    31389
top100       19233
Name: data_source, dtype: int64

In [49]:
audio_features_all_new['track_explicit'].value_counts()

False    33898
True     16724
Name: track_explicit, dtype: int64